The code shown in this notebook is an implementation of training a RESNET18 face recognition model on 43 students

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
import numpy as np
import copy

In [ ]:
import random
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# **ClearML Integration**

In [ ]:
!pip install clearml
import clearml
from clearml import Task

Task.set_credentials(
     api_host="https://api.clear.ml",
     web_host="https://app.clear.ml",
     files_host="https://files.clear.ml",
     key='K87AYXOFYVT60A89H32E',
     secret='xsvMF2rp6TzjXUWZdr6EXQZne7fUUP0feRRM2IC9j425ayd4WA'
)

# Initialize ClearML task
task = Task.init(project_name='Experiments', task_name='raw_data_2')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.0 MB/s eta 0:00:00
ClearML Task: created new task id=b5cf997148c0482a8019bfa76e3d9dbf
2023-06-09 22:38:42,470 - clearml.Task - INFO - Storing jupyter notebook directly as code


ClearML results page: https://app.clear.ml/projects/d82d91dc35834bae98a76c2ada692c85/experiments/b5cf997148c0482a8019bfa76e3d9dbf/output/log


# **ResNet18 Training**

In [ ]:
# Defining the transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
#importing datasets
train_dataset = datasets.ImageFolder('/content/drive/MyDrive/RECOGData/train', transform=transform)
val_dataset = datasets.ImageFolder('/content/drive/MyDrive/RECOGData/val', transform=transform)
test_dataset = datasets.ImageFolder('/content/drive/MyDrive/RECOGData/test', transform=transform)

In [ ]:
# Creating the data loaders
batch_size=32
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 43)
model = model.to(device)


criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

opt = {
    'model_architecture': 'resnet18',
    'learning_rate': 0.001,
    'batch_size': 32,
    'num_epochs': 100,
    'optimizer': 'Adam',
    'loss_function': 'CrossEntropyLoss',
    'train_data_path': '/content/drive/MyDrive/RECOGData/train',
    'val_data_path': '/content/drive/MyDrive/RECOGData/val',
}

#Wrap the model and optimizer with ClearML
opt = task.connect_configuration(opt)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning:

The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning:

Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 189MB/s]


2023-06-09 22:39:52,765 - clearml.model - INFO - Selected model id: 85b1cde9736a4fa98bc157d942969307


In [ ]:
# Train the model
num_epochs = 100
best_loss = float('inf') # Initialize best_loss to infinity
best_model_wts = None

for epoch in range(num_epochs):
    train_correct = 0
    train_total = 0
    train_loss = 0
    model.train()
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()
    train_accuracy = 100 * train_correct / train_total
    train_loss /= len(train_loader)

    task.get_logger().report_scalar(title='Train Accuracy', series='accuracy', value=train_accuracy, iteration=epoch)
    task.get_logger().report_scalar(title='Train Loss', series='loss', value=train_loss, iteration=epoch)

    model.eval()
    with torch.no_grad():
        val_correct = 0
        val_total = 0
        val_loss = 0
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
            loss = criterion(outputs, labels)
            val_loss += loss.item()
        val_accuracy = 100 * val_correct / val_total

        task.get_logger().report_scalar(title='Validation Accuracy', series='accuracy', value=val_accuracy, iteration=epoch)
        task.get_logger().report_scalar(title='Validation Loss', series='loss', value=val_loss, iteration=epoch)


        if val_loss < best_loss:
            best_loss = val_loss
            best_weights = model.state_dict()
            task.upload_artifact(name='model_best', artifact_object='/content/Last_Recognition_Weights_2.pt')
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(best_model_wts, '/content/drive/MyDrive/Last_Recognition_Weights_2.pt')

        print('Epoch [{}/{}], Training Loss: {:.4f}, Training Accuracy: {:.2f}%, Validation Loss: {:.4f}, Validation Accuracy: {:.2f}%'
              .format(epoch+1, num_epochs, train_loss, train_accuracy, val_loss, val_accuracy))


Epoch [1/100], Training Loss: 1.5699, Training Accuracy: 57.16%, Validation Loss: 36.5053, Validation Accuracy: 59.65%
Epoch [2/100], Training Loss: 0.6136, Training Accuracy: 82.60%, Validation Loss: 47.3861, Validation Accuracy: 54.07%
Epoch [3/100], Training Loss: 0.3560, Training Accuracy: 90.05%, Validation Loss: 36.2328, Validation Accuracy: 63.60%
Epoch [4/100], Training Loss: 0.2318, Training Accuracy: 93.26%, Validation Loss: 29.0376, Validation Accuracy: 70.23%
Epoch [5/100], Training Loss: 0.1694, Training Accuracy: 94.93%, Validation Loss: 30.7059, Validation Accuracy: 71.98%
Epoch [6/100], Training Loss: 0.0719, Training Accuracy: 98.23%, Validation Loss: 29.2432, Validation Accuracy: 72.91%
Epoch [7/100], Training Loss: 0.0493, Training Accuracy: 98.72%, Validation Loss: 26.9968, Validation Accuracy: 75.12%
Epoch [8/100], Training Loss: 0.0867, Training Accuracy: 97.56%, Validation Loss: 34.2459, Validation Accuracy: 69.07%
Epoch [9/100], Training Loss: 0.0982, Training A

# **Model Testing**

In [ ]:
#testing the model
model.eval()
with torch.no_grad():
    predicted_labels = []
    true_labels = [] #
    for images, labels in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        predicted_labels.extend(predicted.tolist())
        true_labels.extend(labels.tolist()) #

    correct = sum(1 for p, t in zip(predicted_labels, true_labels) if p == t)
    total = len(predicted_labels)
    accuracy = correct / total * 100

# Print the predicted labels
#we can print the testing accuracy if we want
print(f"Testing accuracy: {accuracy:.2f}%")
print(predicted_labels)

Testing accuracy: 73.49%
[0, 0, 29, 29, 29, 29, 39, 0, 39, 33, 0, 33, 0, 0, 0, 0, 0, 0, 0, 0, 16, 28, 1, 16, 1, 22, 1, 20, 20, 20, 1, 7, 34, 1, 1, 1, 8, 1, 1, 1, 2, 41, 2, 2, 2, 2, 0, 21, 26, 39, 21, 26, 2, 2, 39, 2, 2, 38, 40, 2, 3, 3, 17, 3, 3, 17, 42, 17, 17, 17, 17, 8, 18, 6, 40, 22, 24, 2, 40, 42, 4, 31, 4, 4, 4, 38, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 9, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 22, 6, 6, 6, 6, 6, 6, 6, 6, 13, 7, 7, 7, 7, 9, 7, 7, 7, 7, 7, 7, 7, 7, 14, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 14, 10, 10, 16, 10, 10, 10, 10, 30, 10, 16, 10, 10, 10, 10, 9, 39, 7, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 30, 11, 29, 20, 29, 11, 11, 11, 11, 11, 11, 11, 20, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 18, 40, 18, 13, 18, 3, 8, 8, 8, 22, 3, 8, 18, 40, 4, 15, 14, 14, 14, 14, 14,

# **Testing Using Voting System**

In [12]:
import statistics

def replace_with_mode(data):
    mode_sequence = []
    for i in range(0, len(data), 20):
        subset = data[i:i+20]
        mode = statistics.mode(subset)
        mode_sequence += [mode] * 20

    return mode_sequence

def calculate_accuracy(array1, array2):
    if len(array1) != len(array2):
        raise ValueError("Arrays must have the same length.")

    total_elements = len(array1)
    correct_predictions = sum(a == b for a, b in zip(array1, array2))
    accuracy = correct_predictions / total_elements

    return accuracy

newpred = replace_with_mode(predicted_labels)
accuracy = calculate_accuracy(true_labels, newpred)
print(f"Testing accuracy: {accuracy*100:.2f}%")

Testing accuracy: 90.70%
